In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import KFold, train_test_split
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('/content/drive/MyDrive/NNFL-Assignment1/data_q4_q5.csv')
df = df.replace({'diagnosis': {'B': 0, 'M': 1}})
df.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,1
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,1
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,1
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,1


In [4]:
df = df.drop_duplicates(keep='first')
df = df.sample(frac = 1).reset_index(drop=True)
m = df.shape[0]
n = df.shape[1] - 1
print(m,n)
df.head()

569 30


,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
0,11.06,14.83,70.31,378.2,0.07741,0.04768,0.027120,0.007246,0.1535,0.06214,0.1855,0.6881,1.263,12.98,0.004259,0.014690,0.019400,0.004168,0.01191,0.003537,12.68,20.35,80.79,496.7,0.1120,0.18790,0.20790,0.05556,0.2590,0.09158,0
1,11.89,17.36,76.20,435.6,0.12250,0.07210,0.059290,0.074040,0.2015,0.05875,0.6412,2.2930,4.021,48.84,0.014180,0.014890,0.012670,0.019100,0.02678,0.003002,12.40,18.99,79.46,472.4,0.1359,0.08368,0.07153,0.08946,0.2220,0.06033,0
2,10.08,15.11,63.76,317.5,0.09267,0.04695,0.001597,0.002404,0.1703,0.06048,0.4245,1.2680,2.680,26.43,0.014390,0.012000,0.001597,0.002404,0.02538,0.003470,11.87,21.18,75.39,437.0,0.1521,0.10190,0.00692,0.01042,0.2933,0.07697,0
3,12.18,20.52,77.22,458.7,0.08013,0.04038,0.023830,0.017700,0.1739,0.05677,0.1924,1.5710,1.183,14.68,0.005080,0.006098,0.010690,0.006797,0.01447,0.001532,13.34,32.84,84.58,547.8,0.1123,0.08862,0.11450,0.07431,0.2694,0.06878,0
4,13.37,16.39,86.10,553.5,0.07115,0.07325,0.080920,0.028000,0.1422,0.05823,0.1639,1.1400,1.223,14.66,0.005919,0.032700,0.049570,0.010380,0.01208,0.004076,14.26,22.75,91.99,632.1,0.1025,0.25310,0.33080,0.08978,0.2048,0.07628,0


In [5]:
datan = df.values
X = datan[:, 0:30]
#X = (X - np.min(X, axis = 0))/(np.max(X, axis=0)-np.min(X, axis=0))
X = (X - X.mean(axis = 0))/(X.std(axis = 0))
X = np.append(np.ones([m,1]), X, axis = 1)

y = datan[:, -1]# M -> 1, B -> 0  

w = np.zeros(n+1)
print(X.shape, y.shape)

(569, 31) (569,)


In [6]:
num_classes = len(np.unique(y))
print(num_classes)

2


In [7]:
def plot_gd(costs):
  plt.title('Cost Function vs epochs')
  plt.xlabel('Iterations')
  plt.ylabel('Cost')
  plt.plot(costs)
  plt.show()

In [8]:
def sigmoid(X, w):
  z = np.dot(X, w.T)
  return 1/(1+np.exp(-z.astype(float)))

In [9]:
def cost_L1(X, y, w, lm):
  m = len(y)
  h = sigmoid(X, w)
  return (-1/m)*np.sum(np.multiply(y, np.log(h))+ np.multiply(1-y, np.log(1-h))) + lm*np.sum(abs(w))

In [10]:
def cost_L2(X, y, w, lm):
  m = len(y)
  h = sigmoid(X, w)
  return (-1/m)*np.sum(np.multiply(y, np.log(h))+ np.multiply(1-y, np.log(1-h))) + (lm/(2))*np.sum((w)**2)

In [11]:
def bgd_L2(X, y, w, alpha, T, lm):
  costs = np.zeros(T)
  weights = np.zeros((T,31))
  m = len(y)
  for i in range(T):
    h = sigmoid(X, w)
    w = w*(1-alpha*lm) - (alpha/m)*np.dot(h-y,X)
    costs[i] = cost_L2(X, y, w, lm)
    weights[i, :] = w
  return costs, weights, w
################################################
def sgd_L2(X, y, w, alpha, T, lm):
    costs = np.zeros(T)
    weights = np.zeros((T,31))
    m = len(y)
    for i in range(T):
        rand_index = np.random.randint(m)
        ind_x = X[rand_index:rand_index+1]
        ind_y = y[rand_index:rand_index+1]
        z = ind_x.dot(w)
        h = 1/(1+np.exp(-z.astype(float)))
        w = w*(1-alpha*lm) - alpha * (ind_x.T.dot(h - ind_y))
        costs[i] = cost_L2(ind_x,ind_y,w, lm)
        weights[i, :] = w
    return costs, weights, w
################################################
def mbgd_L2(X, y, w, alpha, T, batch_size, lm):
    costs = np.zeros(T)
    weights = np.zeros((T,31))
    m = len(y)
    for i in range(T):
        rand_index = np.random.randint(m)
        ind_x = X[rand_index:rand_index+batch_size]
        ind_y = y[rand_index:rand_index+batch_size]
        z = ind_x.dot(w)
        h = 1/(1+np.exp(-z.astype(float)))
        w = w*(1-alpha*lm) - (alpha/batch_size) * (ind_x.T.dot(h - ind_y))
        costs[i] = cost_L2(ind_x,ind_y,w, lm)
        weights[i, :] = w
    return costs, weights, w

In [12]:
def bgd_L1(X, y, w, alpha, T, lm):
  costs = np.zeros(T)
  weights = np.zeros((T,31))
  m = len(y)
  for i in range(T):
    h = sigmoid(X, w)
    w = w - (alpha*lm/2)*(np.sign(w)) - (alpha/m)*np.dot(h-y,X)
    costs[i] = cost_L1(X, y, w, lm)
    weights[i, :] = w
  return costs, weights, w
################################################
def sgd_L1(X, y, w, alpha, T, lm):
    costs = np.zeros(T)
    weights = np.zeros((T,31))
    m = len(y)
    for i in range(T):
        rand_index = np.random.randint(m)
        ind_x = X[rand_index:rand_index+1]
        ind_y = y[rand_index:rand_index+1]
        z = ind_x.dot(w)
        h = 1/(1+np.exp(-z.astype(float)))
        w = w - (alpha*lm/2)*(np.sign(w)) - alpha * (ind_x.T.dot(h - ind_y))
        costs[i] = cost_L1(ind_x,ind_y,w, lm)
        weights[i, :] = w
    return costs, weights, w
################################################
def mbgd_L1(X, y, w, alpha, T, batch_size, lm):
    costs = np.zeros(T)
    weights = np.zeros((T,31))
    m = len(y)
    for i in range(T):
        rand_index = np.random.randint(m)
        ind_x = X[rand_index:rand_index+batch_size]
        ind_y = y[rand_index:rand_index+batch_size]
        z = ind_x.dot(w)
        h = 1/(1+np.exp(-z.astype(float)))
        w = w - (alpha*lm/2)*(np.sign(w)) - (alpha/batch_size) * (ind_x.T.dot(h - ind_y))
        costs[i] = cost_L1(ind_x,ind_y,w, lm)
        weights[i, :] = w
    return costs, weights, w

In [13]:
def grid_search_bgd_L1(X_train, X_val, y_train, y_val, w, alpha_vals, T, lm_vals):
  alpha_vals
  acc_max = 0
  alpha_best, lm_best = 0, 0
  for i in alpha_vals:
    for j in lm_vals:
      costs_bgd_L1, weights_bgd_L1, w_bgd_L1 = bgd_L1(X_train, y_train, w, i, T, j)
      predictions = np.round_(sigmoid(X_val, w_bgd_L1))
      TN, TP, FN, FP = 0, 0, 0, 0
      for k in range(len(y_val)):
        if (predictions[k] == 0 and y_val[k] == 0):
          TN = TN + 1
        elif (predictions[k] == 1 and y_val[k] == 1):
          TP = TP + 1
        elif (predictions[k] == 0 and y_val[k] == 1):
          FN = FN + 1
        elif (predictions[k] == 1 and y_val[k] == 0):
          FP = FP + 1
      acc = (TP + TN)/(TP + TN + FN + FP)

      if(acc > acc_max):
        acc_max = acc
        alpha_best = i
        lm_best = j
  return alpha_best, lm_best

def grid_search_sgd_L1(X_train, X_val, y_train, y_val, w, alpha_vals, T, lm_vals):
  acc_max = 0
  alpha_best, lm_best = 0, 0
  for i in alpha_vals:
    for j in lm_vals:
      costs_sgd_L1, weights_sgd_L1, w_sgd_L1 = sgd_L1(X_train, y_train, w, i, T, j)
      predictions = np.round_(sigmoid(X_val, w_sgd_L1))
      TN, TP, FN, FP = 0, 0, 0, 0
      for k in range(len(y_val)):
        if (predictions[k] == 0 and y_val[k] == 0):
          TN = TN + 1
        elif (predictions[k] == 1 and y_val[k] == 1):
          TP = TP + 1
        elif (predictions[k] == 0 and y_val[k] == 1):
          FN = FN + 1
        elif (predictions[k] == 1 and y_val[k] == 0):
          FP = FP + 1
      acc = (TP + TN)/(TP + TN + FN + FP)

      if(acc > acc_max):
        acc_max = acc
        alpha_best = i
        lm_best = j
  return alpha_best, lm_best

def grid_search_mbgd_L1(X_train, X_val, y_train, y_val, w, alpha_vals, T, batch_size, lm_vals):
  acc_max = 0
  alpha_best, lm_best = 0, 0
  for i in alpha_vals:
    for j in lm_vals:
      costs_mbgd_L1, weights_mbgd_L1, w_mbgd_L1 = mbgd_L1(X_train, y_train, w, i, T, batch_size, j)
      predictions = np.round_(sigmoid(X_val, w_mbgd_L1))
      TN, TP, FN, FP = 0, 0, 0, 0
      for k in range(len(y_val)):
        if (predictions[k] == 0 and y_val[k] == 0):
          TN = TN + 1
        elif (predictions[k] == 1 and y_val[k] == 1):
          TP = TP + 1
        elif (predictions[k] == 0 and y_val[k] == 1):
          FN = FN + 1
        elif (predictions[k] == 1 and y_val[k] == 0):
          FP = FP + 1
      acc = (TP + TN)/(TP + TN + FN + FP)

      if(acc > acc_max):
        acc_max = acc
        alpha_best = i
        lm_best = j
  return alpha_best, lm_best

In [14]:
def grid_search_bgd_L2(X_train, X_val, y_train, y_val, w, alpha_vals, T, lm_vals):
  acc_max = 0
  alpha_best = 0
  lm_best = 0
  for i in alpha_vals:
    for j in lm_vals:
      costs_bgd_L2, weights_bgd_L2, w_bgd_L2 = bgd_L2(X_train, y_train, w, i, T, j)
      predictions = np.round_(sigmoid(X_val, w_bgd_L2))
      TN, TP, FN, FP = 0, 0, 0, 0
      for k in range(len(y_val)):
        if (predictions[k] == 0 and y_val[k] == 0):
          TN = TN + 1
        elif (predictions[k] == 1 and y_val[k] == 1):
          TP = TP + 1
        elif (predictions[k] == 0 and y_val[k] == 1):
          FN = FN + 1
        elif (predictions[k] == 1 and y_val[k] == 0):
          FP = FP + 1
      acc = (TP + TN)/(TP + TN + FN + FP)

      if(acc > acc_max):
        acc_max = acc
        alpha_best = i
        lm_best = j
  return alpha_best, lm_best

def grid_search_sgd_L2(X_train, X_val, y_train, y_val, w, alpha_vals, T, lm_vals):
  acc_max = 0
  alpha_best, lm_best = 0, 0
  for i in alpha_vals:
    for j in lm_vals:
      costs_sgd_L2, weights_sgd_L2, w_sgd_L2 = sgd_L2(X_train, y_train, w, i, T, j)
      predictions = np.round_(sigmoid(X_val, w_sgd_L2))
      TN, TP, FN, FP = 0, 0, 0, 0
      for k in range(len(y_val)):
        if (predictions[k] == 0 and y_val[k] == 0):
          TN = TN + 1
        elif (predictions[k] == 1 and y_val[k] == 1):
          TP = TP + 1
        elif (predictions[k] == 0 and y_val[k] == 1):
          FN = FN + 1
        elif (predictions[k] == 1 and y_val[k] == 0):
          FP = FP + 1
      acc = (TP + TN)/(TP + TN + FN + FP)

      if(acc > acc_max):
        acc_max = acc
        alpha_best = i
        lm_best = j
  return alpha_best, lm_best

def grid_search_mbgd_L2(X_train, X_val, y_train, y_val, w, alpha_vals, T, batch_size, lm_vals):
  acc_max = 0
  alpha_best, lm_best = 0, 0
  for i in alpha_vals:
    for j in lm_vals:
      costs_mbgd_L2, weights_mbgd_L2, w_mbgd_L2 = mbgd_L2(X_train, y_train, w, i, T, batch_size, j)
      predictions = np.round_(sigmoid(X_val, w_mbgd_L2))
      TN, TP, FN, FP = 0, 0, 0, 0
      for k in range(len(y_val)):
        if (predictions[k] == 0 and y_val[k] == 0):
          TN = TN + 1
        elif (predictions[k] == 1 and y_val[k] == 1):
          TP = TP + 1
        elif (predictions[k] == 0 and y_val[k] == 1):
          FN = FN + 1
        elif (predictions[k] == 1 and y_val[k] == 0):
          FP = FP + 1
      acc = (TP + TN)/(TP + TN + FN + FP)

      if(acc > acc_max):
        acc_max = acc
        alpha_best = i
        lm_best = j
  return alpha_best, lm_best

In [15]:
def confusion_matrix(predictions, labels):
  TP = 0
  FN = 0
  FP = 0
  TN = 0
  m = len(labels)

  for i in range(m):
      if labels[i] == 0 and predictions[i] == 0:
              TN = TN + 1
      if labels[i] == 1 and predictions[i] == 0:
              FN = FN + 1
      if labels[i] == 0 and predictions[i] == 1:
              FP = FP + 1
      if predictions[i] == 1 and labels[i] == 1:
              TP = TP + 1    
  return TP, FN, FP, TN

In [16]:
alpha_vals = np.linspace(0.01, 0.1, 35).tolist()
lm_vals = np.linspace(0.01, 0.1, 10).tolist()

**LOR + BGD +L1**

In [17]:
kf = KFold(n_splits=5)
iterations = 500
spec_bgd_L1 = []
sens_bgd_L1 = []
acc_bgd_L1 = []
for train_index, test_index in kf.split(X):
  X_train_val, X_test = X[train_index], X[test_index]
  y_train_val, y_test = y[train_index], y[test_index]
  X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)
  alpha_best_bgd_L1, lm_best_bgd_L1 = grid_search_bgd_L1(X_train, X_val, y_train, y_val, w, alpha_vals, iterations, lm_vals)
  costs_bgd_L1, weights_bgd_L1, w_bgd_L1 = bgd_L1(X_train, y_train, w, alpha_best_bgd_L1, iterations, lm_best_bgd_L1)
  pred_bgd_L1 = np.round_(sigmoid(X_test, w_bgd_L1))
  TP, FN, FP, TN = confusion_matrix(pred_bgd_L1,y_test)
  sens_bgd_L1.append(TP/(TP+FN))
  spec_bgd_L1.append(TN/(TN+FP))
  acc_bgd_L1.append((TP+TN)/(TP+TN+FP+FN))

print(sum(sens_bgd_L1)/len(sens_bgd_L1))
print(sum(spec_bgd_L1)/len(spec_bgd_L1))
print(sum(acc_bgd_L1)/len(acc_bgd_L1))

0.9575121705556487
0.98316100128587
0.973591057289241


**LOR + BGD +L2**

In [18]:
kf = KFold(n_splits=5)
iterations = 500
spec_bgd_L2 = []
sens_bgd_L2 = []
acc_bgd_L2 = []
for train_index, test_index in kf.split(X):
  X_train_val, X_test = X[train_index], X[test_index]
  y_train_val, y_test = y[train_index], y[test_index]
  X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)
  alpha_best_bgd_L2, lm_best_bgd_L2 = grid_search_bgd_L2(X_train, X_val, y_train, y_val, w, alpha_vals, iterations, lm_vals)
  costs_bgd_L2, weights_bgd_L2, w_bgd_L2 = bgd_L2(X_train, y_train, w, alpha_best_bgd_L2, iterations, lm_best_bgd_L2)
  pred_bgd_L2 = np.round_(sigmoid(X_test, w_bgd_L2))
  TP, FN, FP, TN = confusion_matrix(pred_bgd_L2,y_test)
  sens_bgd_L2.append(TP/(TP+FN))
  spec_bgd_L2.append(TN/(TN+FP))
  acc_bgd_L2.append((TP+TN)/(TP+TN+FP+FN))

print(sum(sens_bgd_L2)/len(sens_bgd_L2))
print(sum(spec_bgd_L2)/len(spec_bgd_L2))
print(sum(acc_bgd_L2)/len(acc_bgd_L2))

0.9529667160101942
0.9803440998774192
0.9700822853594163


**LOR + SGD +L1**

In [19]:
kf = KFold(n_splits=5)
iterations = 500
spec_sgd_L1 = []
sens_sgd_L1 = []
acc_sgd_L1 = []
for train_index, test_index in kf.split(X):
  X_train_val, X_test = X[train_index], X[test_index]
  y_train_val, y_test = y[train_index], y[test_index]
  X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)
  alpha_best_sgd_L1, lm_best_sgd_L1 = grid_search_sgd_L1(X_train, X_val, y_train, y_val, w, alpha_vals, iterations, lm_vals)
  costs_sgd_L1, weights_sgd_L1, w_sgd_L1 = sgd_L1(X_train, y_train, w, alpha_best_sgd_L1, iterations, lm_best_sgd_L1)
  pred_sgd_L1 = np.round_(sigmoid(X_test, w_sgd_L1))
  TP, FN, FP, TN = confusion_matrix(pred_sgd_L1,y_test)
  sens_sgd_L1.append(TP/(TP+FN))
  spec_sgd_L1.append(TN/(TN+FP))
  acc_sgd_L1.append((TP+TN)/(TP+TN+FP+FN))

print(sum(sens_sgd_L1)/len(sens_sgd_L1))
print(sum(spec_sgd_L1)/len(spec_sgd_L1))
print(sum(acc_sgd_L1)/len(acc_sgd_L1))

0.9529667160101942
0.9742823835823625
0.9665579878900792


**LOR + SGD +L2**

In [20]:
kf = KFold(n_splits=5)
iterations = 500
spec_sgd_L2 = []
sens_sgd_L2 = []
acc_sgd_L2 = []
for train_index, test_index in kf.split(X):
  X_train_val, X_test = X[train_index], X[test_index]
  y_train_val, y_test = y[train_index], y[test_index]
  X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)
  alpha_best_sgd_L2, lm_best_sgd_L2 = grid_search_sgd_L2(X_train, X_val, y_train, y_val, w, alpha_vals, iterations, lm_vals)
  costs_sgd_L2, weights_sgd_L2, w_sgd_L2 = sgd_L2(X_train, y_train, w, alpha_best_sgd_L2, iterations, lm_best_sgd_L2)
  pred_sgd_L2 = np.round_(sigmoid(X_test, w_sgd_L2))
  TP, FN, FP, TN = confusion_matrix(pred_sgd_L2,y_test)
  sens_sgd_L2.append(TP/(TP+FN))
  spec_sgd_L2.append(TN/(TN+FP))
  acc_sgd_L2.append((TP+TN)/(TP+TN+FP+FN))

print(sum(sens_sgd_L2)/len(sens_sgd_L2))
print(sum(spec_sgd_L2)/len(spec_sgd_L2))
print(sum(acc_sgd_L2)/len(acc_sgd_L2))

0.9575121705556487
0.9889629773211862
0.9771153547585778


**LOR + MBGD +L1**

In [21]:
kf = KFold(n_splits=5)
iterations = 500
spec_mbgd_L1 = []
sens_mbgd_L1 = []
acc_mbgd_L1 = []
for train_index, test_index in kf.split(X):
  X_train_val, X_test = X[train_index], X[test_index]
  y_train_val, y_test = y[train_index], y[test_index]
  X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)
  alpha_best_mbgd_L1, lm_best_mbgd_L1 = grid_search_mbgd_L1(X_train, X_val, y_train, y_val, w, alpha_vals, iterations, 50, lm_vals)
  costs_mbgd_L1, weights_mbgd_L1, w_mbgd_L1 = mbgd_L1(X_train, y_train, w, alpha_best_mbgd_L1, iterations, 50, lm_best_mbgd_L1)
  pred_mbgd_L1 = np.round_(sigmoid(X_test, w_mbgd_L1))
  TP, FN, FP, TN = confusion_matrix(pred_mbgd_L1,y_test)
  sens_mbgd_L1.append(TP/(TP+FN))
  spec_mbgd_L1.append(TN/(TN+FP))
  acc_mbgd_L1.append((TP+TN)/(TP+TN+FP+FN))

print(sum(sens_mbgd_L1)/len(sens_mbgd_L1))
print(sum(spec_mbgd_L1)/len(spec_mbgd_L1))
print(sum(acc_mbgd_L1)/len(acc_mbgd_L1))

0.9529667160101942
0.9775663220996413
0.9683278993945039


**LOR + MBGD + L2**

In [22]:
kf = KFold(n_splits=5)
iterations = 500
spec_mbgd_L2 = []
sens_mbgd_L2 = []
acc_mbgd_L2 = []
for train_index, test_index in kf.split(X):
  X_train_val, X_test = X[train_index], X[test_index]
  y_train_val, y_test = y[train_index], y[test_index]
  X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)
  alpha_best_mbgd_L2, lm_best_mbgd_L2 = grid_search_mbgd_L2(X_train, X_val, y_train, y_val, w, alpha_vals, iterations, 50, lm_vals)
  costs_mbgd_L2, weights_mbgd_L2, w_mbgd_L2 = mbgd_L2(X_train, y_train, w, alpha_best_mbgd_L2, iterations, 50, lm_best_mbgd_L2)
  pred_mbgd_L2 = np.round_(sigmoid(X_test, w_mbgd_L2))
  TP, FN, FP, TN = confusion_matrix(pred_mbgd_L2,y_test)
  sens_mbgd_L2.append(TP/(TP+FN))
  spec_mbgd_L2.append(TN/(TN+FP))
  acc_mbgd_L2.append((TP+TN)/(TP+TN+FP+FN))

print(sum(sens_mbgd_L2)/len(sens_mbgd_L2))
print(sum(spec_mbgd_L2)/len(spec_mbgd_L2))
print(sum(acc_mbgd_L2)/len(acc_mbgd_L2))

0.9575121705556487
0.9832012427345621
0.9735910572892408
